In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.ticker import MaxNLocator
%matplotlib inline
plt.ion();
import utils_intensity_map as uim
import utils_deck_generation as idg
import netcdf_read_write as nrw
import training_data_generation as tdg
import utils_healpy as uhp
import utils_optimizers as uopt

In [ ]:
diag_dir = "../Data/test_run"

sys_params = tdg.define_system_params(diag_dir)

In [ ]:
def fitness_function(dataset, opt_params):
    target_rms = opt_params["fitness_desired_rms"]
    norm_factor = opt_params["fitness_norm_factor"]
    number_of_timesteps = np.shape(dataset["rms"][:,:])[1]

    rms = np.sqrt(np.sum(dataset["rms"][:,:]**2, axis=1) / float(number_of_timesteps))
    avg_flux = np.sqrt(np.sum(dataset["avg_flux"][:,:]**2, axis=1) / float(number_of_timesteps))
    if opt_params["run_plasma_profile"]:
        target_flux = opt_params["fitness_desired_pressure_mbar"]
        indices = np.where(np.array(avg_flux) > opt_params["fitness_limit_broken_pressure_mbar"])[0]
        print("Fitness function detects broken runs: ", indices)
        avg_flux[indices] = 0.0
    else:
        target_flux = opt_params["fitness_desired_power_per_steradian"]

    maxi_func = np.exp(-(rms/target_rms) + (avg_flux / target_flux) ** 0.25) * (avg_flux / target_flux)**0.01 * norm_factor
    #maxi_func = np.exp(-(rms/target_rms) + (avg_flux / target_flux) ** 0.25) * (avg_flux / target_flux) * norm_factor
    return maxi_func

def fitness_function_time_dependant(dataset, opt_params):
    target_rms = opt_params["fitness_desired_rms"]
    norm_factor = opt_params["fitness_norm_factor"]
    number_of_timesteps = np.shape(dataset["rms"][:,:])[1]

    rms = dataset["rms"][:,:]
    avg_flux = dataset["avg_flux"][:,:]
    if opt_params["run_plasma_profile"]:
        target_flux = opt_params["fitness_desired_pressure_mbar"]
        indices = np.where(np.array(avg_flux) > opt_params["fitness_limit_broken_pressure_mbar"])[0]
        print("Fitness function detects broken runs: ", indices)
        avg_flux[indices] = 0.0
    else:
        target_flux = opt_params["fitness_desired_power_per_steradian"]

    maxi_func = np.exp(-(rms/target_rms) + (avg_flux / target_flux) ** 0.25) * (avg_flux / target_flux)**0.01 * norm_factor
    return maxi_func

In [ ]:
dataset, dataset_params, deck_gen_params, facility_spec = idg.load_data_dicts_from_file(sys_params)
opt_params = uopt.define_optimizer_parameters(diag_dir, 0, 0, dataset_params, facility_spec, sys_params)

fitness_temporal = fitness_function_time_dependant(dataset, opt_params)
fitness_overall = fitness_function(dataset, opt_params)

labels = [None] * dataset_params["num_examples"]
for iconfig in range(dataset_params["num_examples"]):
    labels[iconfig] = r"SGO={:.2f} and $R_b$={:.2f} $\mu m$".format(deck_gen_params["beamspot_order"][iconfig,0], deck_gen_params["beamspot_major_radius"][iconfig,0])

In [ ]:
ind_max_fitness = np.argmax(fitness_overall)
print(ind_max_fitness)
print(fitness_overall[ind_max_fitness])

#"""
param1 = deck_gen_params["beamspot_order"][:,0]
label1 = "Super guassian order"
param2 = deck_gen_params["beamspot_major_radius"][:,0]/dataset_params['target_radius']
label2 = r"Beam to target ratio ($R_b/R_t$)"
"""
param1 = deck_gen_params["bandwidth_num_spectral_lines"]
label1 = "Number spectral lines"
param2 = deck_gen_params["bandwidth_percentage_width"][:,0]
label2 = r"Bandwidth ($\delta \lambda / \lambda$)"
"""

semilogy_bool = False
initial_dataset = 36#16#36

In [ ]:
fig1, ax1 = plt.subplots(figsize=(7, 4), dpi=400)

cmap = ax1.scatter(param1, param2, c=fitness_overall,s=100)
fig1.colorbar(cmap, ax=ax1, label="Overall fitness")
if semilogy_bool:
    ax1.semilogy(param1[ind_max_fitness],param2[ind_max_fitness],"rx")
else:
    ax1.plot(param1[ind_max_fitness],param2[ind_max_fitness],"rx")

ax1.set_xlabel(label1)
ax1.set_ylabel(label2)

In [ ]:
number_of_timesteps = np.shape(dataset["rms"])[1]

rms_overall = np.sqrt(np.sum(dataset["rms"]**2, axis=1) / float(number_of_timesteps))
avg_flux_overall = np.sqrt(np.sum(dataset["avg_flux"]**2, axis=1) / float(number_of_timesteps))
print(np.shape(rms_overall), np.shape(rms_overall))

num_samples_per_param = int(np.sqrt(initial_dataset)) #int(np.sqrt(dataset_params["num_examples"]))
rms_overall_reshaped = np.reshape(rms_overall[:initial_dataset],(num_samples_per_param,num_samples_per_param))
rms_temporal_reshaped = np.reshape(dataset["rms"][:initial_dataset],(num_samples_per_param,num_samples_per_param,-1))
avg_flux_overall_reshaped = np.reshape(avg_flux_overall[:initial_dataset],(num_samples_per_param,num_samples_per_param))
avg_flux_temporal_reshaped = np.reshape(dataset["avg_flux"][:initial_dataset],(num_samples_per_param,num_samples_per_param,-1))
fitness_overall_reshaped = np.reshape(fitness_overall[:initial_dataset],(num_samples_per_param,num_samples_per_param))
fitness_temporal_reshaped = np.reshape(fitness_temporal[:initial_dataset],(num_samples_per_param,num_samples_per_param,-1))
print(np.shape(fitness_temporal_reshaped), np.shape(avg_flux_temporal_reshaped))
print(np.shape(dataset["input_parameters"]))

#X = np.reshape(dataset["input_parameters"][:,0],(num_samples_per_param,num_samples_per_param))
#Y = np.reshape(dataset["input_parameters"][:,1],(num_samples_per_param,num_samples_per_param))

X = np.reshape(param1[:initial_dataset],(num_samples_per_param,num_samples_per_param))
Y = np.reshape(param2[:initial_dataset],(num_samples_per_param,num_samples_per_param))
print(np.shape(X), np.shape(Y))
#print(rms_overall)

In [ ]:
fig1, ax1 = plt.subplots(figsize=(7, 4), dpi=400)
fig2, ax2 = plt.subplots(figsize=(7, 4), dpi=400)
fig3, ax3 = plt.subplots(figsize=(7, 4), dpi=400)

cmap = ax1.pcolormesh(X, Y, fitness_overall_reshaped)
fig1.colorbar(cmap, ax=ax1, label="Overall fitness")
if semilogy_bool:
    ax1.semilogy(param1[ind_max_fitness],param2[ind_max_fitness],"rx")
else:
    ax1.plot(param1[ind_max_fitness],param2[ind_max_fitness],"rx")

cmap = ax2.pcolormesh(X, Y, rms_overall_reshaped * 100., norm=matplotlib.colors.LogNorm())
fig2.colorbar(cmap, ax=ax2, label="Overall rms (%)")
if semilogy_bool:
    ax2.semilogy(param1[ind_max_fitness],param2[ind_max_fitness],"rx")
else:
    ax2.plot(param1[ind_max_fitness],param2[ind_max_fitness],"rx")

cmap = ax3.pcolormesh(X, Y, avg_flux_overall_reshaped)
fig3.colorbar(cmap, ax=ax3, label="Overall ablation pressure (Mbars)")
if semilogy_bool:
    ax3.semilogy(param1[ind_max_fitness],param2[ind_max_fitness],"rx")
else:
    ax3.plot(param1[ind_max_fitness],param2[ind_max_fitness],"rx")

ax1.set_xlabel(label1)
ax1.set_ylabel(label2)
ax2.set_xlabel(label1)
ax2.set_ylabel(label2)
ax3.set_xlabel(label1)
ax3.set_ylabel(label2)

In [ ]:
eval_time_ind = 0

fig1, ax1 = plt.subplots(figsize=(7, 4), dpi=400)
fig2, ax2 = plt.subplots(figsize=(7, 4), dpi=400)
fig3, ax3 = plt.subplots(figsize=(7, 4), dpi=400)

cmap = ax1.pcolormesh(X, Y, fitness_temporal_reshaped[:,:,eval_time_ind])
fig1.colorbar(cmap, ax=ax1, label="Time dependant fitness")
if semilogy_bool:
    ax1.semilogy(param1[ind_max_fitness],param2[ind_max_fitness],"rx")
else:
    ax1.plot(param1[ind_max_fitness],param2[ind_max_fitness],"rx")

rms_temporal_reshaped = np.reshape(dataset["rms"][:initial_dataset],(num_samples_per_param,num_samples_per_param,-1))
cmap = ax2.pcolormesh(X, Y, rms_temporal_reshaped[:,:,eval_time_ind] * 100., norm=matplotlib.colors.LogNorm())
fig2.colorbar(cmap, ax=ax2, label="Time dependant rms (%)")
if semilogy_bool:
    ax2.semilogy(param1[ind_max_fitness],param2[ind_max_fitness],"rx")
else:
    ax2.plot(param1[ind_max_fitness],param2[ind_max_fitness],"rx")

cmap = ax3.pcolormesh(X, Y, avg_flux_temporal_reshaped[:,:,eval_time_ind])
fig3.colorbar(cmap, ax=ax3, label="Time dependant ablation pressure (Mbars)")
if semilogy_bool:
    ax3.semilogy(param1[ind_max_fitness],param2[ind_max_fitness],"rx")
else:
    ax3.plot(param1[ind_max_fitness],param2[ind_max_fitness],"rx")

ax1.set_xlabel(label1)
ax1.set_ylabel(label2)
ax2.set_xlabel(label1)
ax2.set_ylabel(label2)
ax3.set_xlabel(label1)
ax3.set_ylabel(label2)

In [ ]:
target_rms = opt_params["fitness_desired_rms"]
norm_factor = opt_params["fitness_norm_factor"]
if (np.mean(avg_flux_overall)>1.e6):
    target_flux = opt_params["fitness_desired_power_per_steradian"] * 3.0
else:
    target_flux = opt_params["fitness_desired_pressure_mbar"]

fig1, ax1 = plt.subplots(figsize=(7, 4), dpi=400)
fig2, ax2 = plt.subplots(figsize=(7, 4), dpi=400)

cut = 10
ax1.semilogy(dataset_params["plasma_profile_times"], fitness_temporal[:cut].T, "x:", label=labels[:cut])
ax1.semilogy(dataset_params["plasma_profile_times"], fitness_temporal[cut:].T, ".", label=labels[cut:])

print(dataset_params["plasma_profile_times"], fitness_temporal[:cut].T)

ax2.plot(fitness_overall, label=labels)
ax2.plot(2.-rms_overall / target_rms, "r:", label=labels)
ax2.plot(avg_flux_overall / target_flux, "r", label=labels)

xlims = [0.,30.0]
ylims = [0.01,10.0]

ax1.set_xlim(xlims)
#ax1.set_ylim(ylims)
ax1.set_xlabel("Time (ns)")
ax1.set_ylabel("Fitness time dependant")
ax1.legend(loc="best")

ylims = [0,2.5]

ax2.set_ylim(ylims)
ax2.set_xlabel("Configuration number")
ax2.set_ylabel("Fitness (rms of all times)")
#ax2.legend(loc="best")

In [ ]:
fig1, ax1 = plt.subplots(figsize=(7, 4), dpi=400)
fig2, ax2 = plt.subplots(figsize=(7, 4), dpi=400)

cut = 10
ax1.semilogy(dataset_params["plasma_profile_times"],dataset["rms"][:cut].T * 100, "x:", label=labels[:cut])
ax1.semilogy(dataset_params["plasma_profile_times"],dataset["rms"][cut:].T * 100, ".", label=labels[cut:])
ax2.plot(dataset_params["plasma_profile_times"][:],dataset["avg_flux"][:cut,:].T, "x:", label=labels[:cut])
ax2.plot(dataset_params["plasma_profile_times"][:],dataset["avg_flux"][cut:,:].T, ".", label=labels[cut:])

xlims = [0.,30.0]
ylims = [0.,100.0]

ax1.set_xlim(xlims)
#ax1.set_ylim(ylims)
ax1.set_xlabel("Time (ns)")
ax1.set_ylabel("RMS ablation pressure (%)")
ax1.legend(loc="best")
#fig1.savefig(root+"/"+plot_dir + "/" + 'hydro_equivalent_intensity.png', bbox_inches='tight')

ax2.set_xlim(xlims)
ax2.set_xlabel("Time (ns)")
ax2.set_ylabel("Ablation pressure (Mbars)")
ax2.legend(loc="best")
#fig2.savefig(root+"/"+plot_dir + "/" + 'hydro_equivalent_ablation_pressure_estimate.png', bbox_inches='tight')